<a href="https://colab.research.google.com/github/Ogirimah/Retraining-a-TensorFlow-Model/blob/main/Retraining_a_Python_model_for_Natural_Language_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tflite-model-maker

In [2]:
import numpy as np
import os
 
from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.text_classifier import DataLoader
 
import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

The code essentially imports useful utilities and Model Maker functions that are needed to set up the spam classifier model. The last section of the code ensures that the TensorFlow version is version 2 because this is required for the model maker to work correctly. In case any issues are found, an error message is logged. 

You now need to add new training data to train your model. For this project, data has been hosted in a ‘.csv’ file (where data is separated by commas) that contains the original training data used for training along with the extra examples to account for the edge cases you discovered earlier on.

The below code downloads the csv file

In [ ]:
data_file = tf.keras.utils.get_file(fname='comment-spam-extras.csv', origin='https://storage.googleapis.com/jmstore/TensorFlowJS/EdX/code/6.5/jm_blog_comments_extras.csv', extract=False)

The ‘tf.keras.utils.get_file’ function is used here to access a remote file and store it locally on the disk. In this case, it stores the file with the filename ‘comment-spam-extras.csv’ locally once downloaded. As this is not a zip file, ‘extract’ is set to false.

Note: Model Maker can train models from simple .csv files like the one you just saw. However, you need to specify the columns that hold the text, and columns that hold the labels which you will see further below how to do this. When using Model Maker, you usually don't build models from scratch, but use existing models to customize to your needs.

In [4]:
spec = model_spec.get('average_word_vec')
spec.num_words = 2000
spec.seq_len = 20
spec.wordvec_dim = 7

Model Maker provides several pre-learned models that you can use, but the simplest and quickest, to begin with, is average_word_vec, which is stored in a variable called ‘spec’. 
Specify the number of words you want the model to use. In this case use 2,000.

Note: It is important to arrive at an appropriate number of words that you want the model to store. If you use every word in the entire corpus, you could end up with the model trying to learn weights for words that are used only once. This will not help you classify future sentences accurately since these words are rarely used. Try to arrive at a reasonable number by analyzing how many words there are and their usage counts. Remember, using fewer words will lead to a smaller and faster model, but may result in a less accurate model if too few are used. This takes experimentation to see what works well with your data.
The next line of code represents the sequence length of the input sentence. This is the number of tokens the model can accept as input. Here, it is set to 20 just like the original pre-trained model.
The last line involves a property called ‘wordvec_dim’, which stands for the number of word vector dimensions that are used to separate words by. These dimensions are the different characteristics, learned by the machine learning algorithm when training, by which any given word can be measured.

The model uses these dimensions to best associate words that are similar in a meaningful way. It uses these dimensions to then detect words that are more likely associated with spam. For instance, it may determine that spam emails are more likely to contain words that are both “medical” in nature and also related to  “human body parts”, so a model may discover that using these two dimensions (medical and body parts) is useful to separate such data to classify well.

Note: A rule of thumb determined from research is that the fourth root of the number of words is appropriate for the number of dimensions. If you are using 2,000 words, a good starting point is 7 dimensions. 

In [5]:
data = DataLoader.from_csv(
      filename=data_file,
      text_column='commenttext', 
      label_column='spam', 
      model_spec=spec,
      delimiter=',',
      shuffle=True,
      is_training=True)
 
train_data, test_data = data.split(0.9)

Load the data from the .csv file you downloaded earlier and use it as training data for the model using the Python ‘DataLoader’ utilities class.

Start by specifying the following parameters:
1. The filename which is simply equal to “data_file” variable you defined earlier.
2. The column in the .csv file where sentences are found; in this case, the name of the column is ‘commenttext’.
3. The column where the labels are found;  in this case, the column name is ‘spam’. 

Note: The names of the columns can be found on line 1 of the .csv file.

4. Next Pass the ‘spec’ variable previously defined with all the configurations set
5. Set the delimiter that the file uses. As this is a .csv file, a comma is used as the separator of columns.
6. Set both ‘shuffle’ and ‘is_training’ values to ‘true’ so that the data gets shuffled and used for training.

Once the data is loaded, you can then split the data into training and testing data sets using ‘data.split’ and pass a value of 0.9. This ensures 90% of the data is reserved for training and 10% for testing.

**Build and Train the Model**

In [ ]:
model = text_classifier.create(train_data, model_spec=spec, epochs=50)

The ‘text_classifier.create’ method is called with three parameters. The first parameter passes the training data, the second parameter passes the model specification, and the third parameter is the number of epochs that the model needs to train for. 
Execute the code in the current cell and wait for the model to train. You will see the epoch data printed as it completes each of the 50 epochs and if all goes to plan the loss should get lower and the accuracy higher over time

**convert and download the model to use in your JavaScript code.**

In [ ]:
model.export(export_dir="/tmp/js_export/", export_format=[ExportFormat.TFJS, ExportFormat.LABEL, ExportFormat.VOCAB])
!zip -r /tmp/js_export/ModelFiles.zip /tmp/js_export/

Call ‘model.export’ with 2 parameters. The first parameter defines the directory where the model should be exported to. Here a ‘js_export’ directory is created in the server’s ‘tmp’ folder. The second parameter specifies the export format. This function takes a further3 parameters. The first specifies it should be exported to the TensorFlow.js model format, the second requests an export of the labels file, and the third exports the vocab file.

Finally, call the zip utility on the command line to download all the resulting files in one go. Note that this line starts with an ‘exclamation’ point since it is not Python code. The resulting zip file is called ModelFiles.zip and will be stored in the ‘/tmp/js_export’ directory. 

Note: You can use any .csv file with your own pairs of sentences and classification values if you wanted. That means you can even retrain this model to detect things beyond just spam if you have the necessary training data for it. For example, the same model could be trained to figure out the language in which a sentence has been written, in this case, you would have example sentences in different languages, and then a label for what it was eg “French, Spanish, German, English” etc. You could then use the model to predict what language a given input sentence was written in by updating the JS code logic you already wrote.


This file zip file can then be uploaded to anywhere a backend TensorFlow model can be used, and used for prediction